In [3]:
import sys
sys.path.append('..')

import pandas as pd
from utils import load_jsonl, dump_jsonl
import numpy as np

In [4]:
def get_task1_conver(in_dir, col_label, skips=[]):
    conversations = load_jsonl(f"{in_dir}/annotated_conersations.jsonl")
    
    
    def to_message_str(messages, users):
        text = ""
        for m in messages:
            
            if users[m['user_id']] == "SYS":
                text += "A: "+ m['text']+"\n"
            else:
                text += "B: "+ m['text']+"\n"
                
        return text
        
    newdata = []
    for idx, row in enumerate(conversations):
        row["messages"].sort(key=lambda x: x["date_created"], reverse=False)
        
        users = {}
        for m in row["messages"]:
            if m["user_id"] not in users:
#                 username = "USR"+str(len(users)+1) if len(users.keys())==0 else "SYS"
                username = "USR" if len(users.keys())==0 else "SYS"
                users[m["user_id"]] = username
                
#         if len(users)>2:
#             print("More than 1 users", len(users))
        
        
        messages = row["messages"]
        
        ##### SPLIT THEM INTO A SMALLER CHUNK SO THEY ARE EASIER TO ANNOTATE #######
        chunk_size = 20
        for i in range(0, len(messages), chunk_size):
            sub_messages = messages[i:i+chunk_size]
            text = to_message_str(sub_messages, users)
            
            
            # !!! this if should be removed !!!
            if pd.isna(row[col_label]):
                continue
            
            if row[col_label] in skips:
                continue
                
            newdata.append({
                "text": text,
#                 "label": label,
                "idx": idx,
                "nturn": len(sub_messages)
            })
        
    n_val = int(len(newdata)*0.05)
    n_test = n_val
    
    test = newdata[0:n_test]
    val = newdata[n_test:n_test+n_val]
    train = newdata[n_test+n_val:]
    
    return pd.DataFrame(newdata).sample(frac=1, random_state=42)

In [9]:
df = get_task1_conver("../../Task1/", "closeness", skips = [])

Loaded 1234 records from ../../Task1//annotated_conersations.jsonl


In [ ]:
# unnotated = []  
# for idx, row in df.iterrows():
#     # print(row["text"])
#     unnotated.append({
#         "text": row["text"],
#         "idx": row["idx"],
#         "label": []
#     })
# dump_jsonl("v3_unannotated.jsonl", unnotated)

2639

In [40]:
# print(unnotated[0]["text"])